In [ ]:
import numpy as np
import torch
import mmengine
from mmdet3d.structures import LiDARInstance3DBoxes

In [ ]:
# info_path = '../data/nuscenes/nuscenes_infos_train.pkl'
info_path = '../data/CODA/coda_infos_train.pkl'
info = mmengine.load(info_path)

In [25]:
info.keys()

dict_keys(['metainfo', 'data_list'])

In [24]:
info['metainfo']

{'categories': {'Car': 0,
  'Pedestrian': 1,
  'Bike': 2,
  'Motorcycle': 3,
  'Golf Cart': 4,
  'Truck': 5,
  'Scooter': 6,
  'Tree': 7,
  'Traffic Sign': 8,
  'Canopy': 9,
  'Traffic Light': 10,
  'Bike Rack': 11,
  'Bollard': 12,
  'Construction Barrier': 13,
  'Parking Kiosk': 14,
  'Mailbox': 15,
  'Fire Hydrant': 16,
  'Freestanding Plant': 17,
  'Pole': 18,
  'Informational Sign': 19,
  'Door': 20,
  'Fence': 21,
  'Railing': 22,
  'Cone': 23,
  'Chair': 24,
  'Bench': 25,
  'Table': 26,
  'Trash Can': 27,
  'Newspaper Dispenser': 28,
  'Room Label': 29,
  'Stanchion': 30,
  'Sanitizer Dispenser': 31,
  'Condiment Dispenser': 32,
  'Vending Machine': 33,
  'Emergency Aid Kit': 34,
  'Fire Extinguisher': 35,
  'Computer': 36,
  'Television': 37,
  'Other': 38,
  'Horse': 39,
  'Pickup Truck': 40,
  'Delivery Truck': 41,
  'Service Vehicle': 42,
  'Utility Vehicle': 43,
  'Fire Alarm': 44,
  'ATM': 45,
  'Cart': 46,
  'Couch': 47,
  'Traffic Arm': 48,
  'Wall Sign': 49,
  'Floor S

In [6]:
info['data_list'][10].keys()

dict_keys(['scene', 'frame', 'sample_idx', 'token', 'timestamp', 'lidar_points', 'images', 'instances', 'ego2global'])

In [23]:
info['data_list'][1000]['ego2global']

array([[-6.97381136e-03,  9.99968145e-01, -3.88258233e-03,
        -1.04656410e+02],
       [-9.98723210e-01, -6.77075691e-03,  5.00610218e-02,
         1.70632961e+02],
       [ 5.00331391e-02,  4.22674122e-03,  9.98738614e-01,
        -2.39736400e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
instance_range = [np.inf, np.inf, np.inf, -np.inf, -np.inf, -np.inf]
corners = []
for sample in info['data_list']:
    bboxes = [instance['bbox_3d'] for instance in sample['instances']]
    bboxes_3d = LiDARInstance3DBoxes(bboxes, origin=(0.5, 0.5, 0.5))
    corner = bboxes_3d.corners.reshape(-1, 3).numpy()
    corners.append(np.concatenate([corner.min(axis=0), corner.max(axis=0)]))
    instance_range[0:3] = np.minimum(instance_range[0:3], np.min(corner, axis=0))
    instance_range[3:6] = np.maximum(instance_range[3:6], np.max(corner, axis=0))

In [ ]:
corners

In [ ]:
instance_range

In [ ]:
info['data_list'][10]['instances']

In [ ]:
info['data_list'][0]['images']['cam1']

In [ ]:
BBOX_CLASS_TO_ID = {
    # Dynamic Classes
    "Car"                   : 0,
    "Pedestrian"            : 1,
    "Bike"                  : 2,
    "Motorcycle"            : 3,
    "Golf Cart"             : 4, # Unused
    "Truck"                 : 5, # Unused
    "Scooter"               : 6,
    # Static Classes
    "Tree"                  : 7,
    "Traffic Sign"          : 8,
    "Canopy"                : 9,
    "Traffic Light"         : 10,
    "Bike Rack"             : 11,
    "Bollard"               : 12,
    "Construction Barrier"  : 13, # Unused
    "Parking Kiosk"         : 14,
    "Mailbox"               : 15,
    "Fire Hydrant"          : 16,
    # Static Class Mixed
    "Freestanding Plant"    : 17,
    "Pole"                  : 18,
    "Informational Sign"    : 19,
    "Door"                  : 20,
    "Fence"                 : 21,
    "Railing"               : 22,
    "Cone"                  : 23,
    "Chair"                 : 24,
    "Bench"                 : 25,
    "Table"                 : 26,
    "Trash Can"             : 27,
    "Newspaper Dispenser"   : 28,
    # Static Classes Indoor
    "Room Label"            : 29,
    "Stanchion"             : 30,
    "Sanitizer Dispenser"   : 31,
    "Condiment Dispenser"   : 32,
    "Vending Machine"       : 33,
    "Emergency Aid Kit"     : 34,
    "Fire Extinguisher"     : 35,
    "Computer"              : 36,
    "Television"            : 37, # unused
    "Other"                 : 38,
    "Horse"                 : 39,
    # New Classes
    "Pickup Truck"          : 40,
    "Delivery Truck"        : 41,
    "Service Vehicle"       : 42,
    "Utility Vehicle"       : 43,
    "Fire Alarm"            : 44,
    "ATM"                   : 45,
    "Cart"                  : 46,
    "Couch"                 : 47,
    "Traffic Arm"           : 48,
    "Wall Sign"             : 49,
    "Floor Sign"            : 50,
    "Door Switch"           : 51,
    "Emergency Phone"       : 52,
    "Dumpster"              : 53,
    "Vacuum Cleaner"        : 54, # unused
    "Segway"                : 55,
    "Bus"                   : 56,
    "Skateboard"            : 57,
    "Water Fountain"        : 58
}

In [ ]:
METAINFO = {
        'classes':
        tuple(BBOX_CLASS_TO_ID.keys()),
        'version':
        'v1.0-trainval',
        'palette': [
            (255, 158, 0),  # Orange
            (255, 99, 71),  # Tomato
            (255, 140, 0),  # Darkorange
            (255, 127, 80),  # Coral
            (233, 150, 70),  # Darksalmon
            (220, 20, 60),  # Crimson
            (255, 61, 99),  # Red
            (0, 0, 230),  # Blue
            (47, 79, 79),  # Darkslategrey
            (112, 128, 144),  # Slategrey
        ]
    }

In [ ]:
METAINFO['classes']